In [ ]:
import imghdr
import glob
import os

for image in glob.glob('dataset/train/*/*.jpg'):
    if imghdr.what(image) != 'jpeg':
        print('removing {}'.format(image))
        os.remove(image)

for image in glob.glob('dataset/val/*/*.jpg'):
    if imghdr.what(image) != 'jpeg':
        print('removing {}'.format(image))
        os.remove(image)

In [1]:
import os

root_path = 'dataset/'
sample_path = 'sample_images/' # path to sample images use for testing the model
labels = os.listdir(root_path+'train')  # names of the classes
n_classes = len(labels) # number of classes
img_width =  img_height = 299 # width and height of input image (must be 299x299)

print("Total classes: {}".format(n_classes))

Total classes: 20


In [2]:
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras.models import Model, Input
from keras.applications import xception

# create the base pre-trained model
base_model = xception.Xception(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer
predictions = Dense(n_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

Using TensorFlow backend.


In [3]:
# freeze all convolutional Xception layers
for layer in base_model.layers:
    layer.trainable = False

model.summary()

0][0]         
__________________________________________________________________________________________________
block5_sepconv3 (SeparableConv2 (None, None, None, 7 536536      block5_sepconv3_act[0][0]        
__________________________________________________________________________________________________
block5_sepconv3_bn (BatchNormal (None, None, None, 7 2912        block5_sepconv3[0][0]            
__________________________________________________________________________________________________
add_4 (Add)                     (None, None, None, 7 0           block5_sepconv3_bn[0][0]         
                                                                 add_3[0][0]                      
__________________________________________________________________________________________________
block6_sepconv1_act (Activation (None, None, None, 7 0           add_4[0][0]                      
______________________________________________________________________________________________

In [4]:
optimizer = RMSprop(lr=0.001, rho=0.9)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=["accuracy"])

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_set = train_datagen.flow_from_directory(
    root_path+'train/',
    target_size=(img_width, img_height),
    batch_size=8,
    class_mode='categorical',)

test_set = test_datagen.flow_from_directory(
    root_path+'val',
    target_size=(img_width, img_height),
    batch_size=8,
    class_mode='categorical',)

Found 1873 images belonging to 20 classes.
Found 477 images belonging to 20 classes.


In [6]:
from keras.callbacks import ModelCheckpoint
my_callback = [ModelCheckpoint('cats_and_dogs.h5',save_best_only=True,monitor='val_accuracy')]

In [7]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [8]:
hist = model.fit_generator(
    train_set,
    steps_per_epoch= train_set.samples // train_set.batch_size,
    epochs=5,
    validation_data=test_set,
    validation_steps=test_set.samples // test_set.batch_size,
    callbacks = my_callback)

Epoch 1/5
234/234 [==============================] - 256s 1s/step - loss: 1.0065 - accuracy: 0.7035 - val_loss: 0.5078 - val_accuracy: 0.7627
Epoch 2/5
234/234 [==============================] - 245s 1s/step - loss: 0.7512 - accuracy: 0.7979 - val_loss: 0.6626 - val_accuracy: 0.8081
Epoch 3/5
234/234 [==============================] - 223s 952ms/step - loss: 0.7196 - accuracy: 0.8177 - val_loss: 0.4831 - val_accuracy: 0.8081
Epoch 4/5
234/234 [==============================] - 213s 911ms/step - loss: 0.7145 - accuracy: 0.8241 - val_loss: 0.7732 - val_accuracy: 0.6546
Epoch 5/5
234/234 [==============================] - 239s 1s/step - loss: 0.6472 - accuracy: 0.8413 - val_loss: 0.3366 - val_accuracy: 0.8188


In [9]:
bst_val_acc = max(hist.history['val_accuracy'])
print("Best val_accuracy: {:.1%}".format(bst_val_acc))

Best val_accuracy: 81.9%


In [16]:
# we chose to train the top 2 xception blocks, i.e. we will freeze the first 116 layers and unfreeze the rest:
for layer in model.layers[:36]:
    layer.trainable = False
for layer in model.layers[36:]:
    layer.trainable = True

In [17]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=["accuracy"])

In [21]:
hist = model.fit_generator(
    train_set,
    steps_per_epoch= train_set.samples // train_set.batch_size,
    epochs=5,
    validation_data=test_set,
    validation_steps=test_set.samples // test_set.batch_size,
    callbacks=my_callback)

Epoch 1/5
234/234 [==============================] - 411s 2s/step - loss: 0.2460 - accuracy: 0.9239 - val_loss: 0.0107 - val_accuracy: 0.9364
Epoch 2/5
234/234 [==============================] - 443s 2s/step - loss: 0.2312 - accuracy: 0.9282 - val_loss: 0.2481 - val_accuracy: 0.9382
Epoch 3/5
234/234 [==============================] - 410s 2s/step - loss: 0.2439 - accuracy: 0.9249 - val_loss: 0.0090 - val_accuracy: 0.9382
Epoch 4/5
234/234 [==============================] - 410s 2s/step - loss: 0.2268 - accuracy: 0.9340 - val_loss: 0.9261 - val_accuracy: 0.9360
Epoch 5/5
234/234 [==============================] - 409s 2s/step - loss: 0.2009 - accuracy: 0.9399 - val_loss: 7.3092e-04 - val_accuracy: 0.9382


In [22]:
bst_val_acc = max(hist.history['val_accuracy'])
print("Best val_accuracy: {:.1%}".format(bst_val_acc))

Best val_accuracy: 93.8%


## Save the trained model

In [23]:
model.save('cats_and_dogs.h5')

## Dump classes dictionary to pickle file

In [15]:
import pickle
classes = train_set.class_indices
f = open('dict.pkl','wb')
pickle.dump(classes,f)
f.close()